In [94]:
# Imports

import pandas as pd
import numpy as np
import os

In [95]:
# Define the folder path
data_path = "data/"

# Load datasets
osebx = pd.read_csv(os.path.join(data_path, "osebx_prices.csv"), delimiter=";")
zero_coupon = pd.read_csv(os.path.join(data_path, "zero_coupon_rates.csv"), delimiter=";")
exchange_rates = pd.read_csv(os.path.join(data_path, "usd_nok.csv"), delimiter=";")
inflation = pd.read_csv(os.path.join(data_path, "kpi.csv"), delimiter=";")


In [96]:
# Check missing values
print(osebx.isnull().sum())
print(zero_coupon.isnull().sum())
print(exchange_rates.isnull().sum())
print(inflation.isnull().sum())


Date      0
Close     0
High      0
Low       0
Open      0
Volume    0
dtype: int64
FREQ           0
Frequency      0
TENOR          0
Tenor          0
DECIMALS       0
TIME_PERIOD    0
OBS_VALUE      0
dtype: int64
FREQ                    0
Frequency               0
BASE_CUR                0
Base Currency           0
QUOTE_CUR               0
Quote Currency          0
TENOR                   0
Tenor                   0
DECIMALS                0
CALCULATED              0
UNIT_MULT               0
Unit Multiplier         0
COLLECTION              0
Collection Indicator    0
TIME_PERIOD             0
OBS_VALUE               0
dtype: int64
Date    0
kpi     0
dtype: int64


No missing values

In [97]:
# Convert Date to Datetime format
osebx["Date"] = pd.to_datetime(osebx["Date"])
zero_coupon["TIME_PERIOD"] = pd.to_datetime(zero_coupon["TIME_PERIOD"])
exchange_rates["TIME_PERIOD"] = pd.to_datetime(exchange_rates["TIME_PERIOD"])
inflation["Date"] = pd.to_datetime(inflation["Date"], format="%YM%m")

print(inflation.head())

        Date    kpi
0 2015-03-01   99,2
1 2015-04-01   99,6
2 2015-05-01   99,8
3 2015-06-01  100,1
4 2015-07-01  100,1


In [98]:
# Create a full date range from the first to last available date in your dataset
full_date_range = pd.date_range(start=inflation["Date"].min(), end=inflation["Date"].max(), freq="D")

# Create a DataFrame with daily dates
inflation_daily = pd.DataFrame({"Date": full_date_range})

# Merge with the original inflation data (left join) and forward-fill missing values
inflation_daily = inflation_daily.merge(inflation, on="Date", how="left")
inflation_daily["kpi"] = inflation_daily["kpi"].fillna(method="ffill")  # Forward-fill missing values

print(inflation_daily.head())  # Check results


        Date   kpi
0 2015-03-01  99,2
1 2015-03-02  99,2
2 2015-03-03  99,2
3 2015-03-04  99,2
4 2015-03-05  99,2


/var/folders/p_/8fbq4ggx22xdlng4fj8s59mh0000gn/T/ipykernel_1195/3377985114.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  inflation_daily["kpi"] = inflation_daily["kpi"].fillna(method="ffill")  # Forward-fill missing values


In [99]:
# Compute log returns
osebx["log_return"] = np.log(osebx["Close"] / osebx["Close"].shift(1))
osebx.dropna(inplace=True)  # Drop the first row where return cannot be calculated

# Rename columns to "Date"
zero_coupon.rename(columns={"TIME_PERIOD": "Date"}, inplace=True)
exchange_rates.rename(columns={"TIME_PERIOD": "Date"}, inplace=True)

# Merge all datasets on 'Date'
df = osebx.merge(zero_coupon, on="Date", how="inner")
df = df.merge(exchange_rates, on="Date", how="inner")
df = df.merge(inflation_daily, on="Date", how="inner") 

print(df.head())  # Check final dataset

        Date       Close        High         Low        Open  Volume  \
0 2015-03-03  616.919983  617.729980  613.419983  613.630005       0   
1 2015-03-04  609.250000  616.909973  606.840027  616.890015       0   
2 2015-03-05  616.599976  616.849976  609.250000  609.250000       0   
3 2015-03-06  618.309998  621.460022  616.460022  616.469971       0   
4 2015-03-09  614.260010  618.419983  613.049988  618.299988       0   

   log_return FREQ_x Frequency_x TENOR_x  ... TENOR_y  Tenor_y  DECIMALS_y  \
0    0.005967      B    Business     10Y  ...      SP     Spot           4   
1   -0.012511      B    Business     10Y  ...      SP     Spot           4   
2    0.011992      B    Business     10Y  ...      SP     Spot           4   
3    0.002769      B    Business     10Y  ...      SP     Spot           4   
4   -0.006572      B    Business     10Y  ...      SP     Spot           4   

  CALCULATED UNIT_MULT Unit Multiplier COLLECTION  \
0      False         0           Units       

In [100]:
df = df.drop(columns=["Close", "High", "Low", "Open", "Volume",
       "FREQ_x", "Frequency_x", "TENOR_x", "Tenor_x", "DECIMALS_x",
       "FREQ_y", "Frequency_y", "BASE_CUR", "Base Currency",
       "QUOTE_CUR", "Quote Currency", "TENOR_y", "Tenor_y", "DECIMALS_y",
       "CALCULATED", "UNIT_MULT", "Unit Multiplier", "COLLECTION",
       "Collection Indicator"])

df.rename(columns={"OBS_VALUE_x": "zero_coupon_rate"}, inplace=True)
df.rename(columns={"OBS_VALUE_y": "usd_nok_exchange_rate"}, inplace=True)

print(df.head())

        Date  log_return  zero_coupon_rate  usd_nok_exchange_rate   kpi
0 2015-03-03    0.005967             1.540                 7.7091  99,2
1 2015-03-04   -0.012511             1.546                 7.7441  99,2
2 2015-03-05    0.011992             1.567                 7.7207  99,2
3 2015-03-06    0.002769             1.639                 7.7917  99,2
4 2015-03-09   -0.006572             1.676                 7.8992  99,2
